In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os
import time
from datetime import datetime, timedelta
import pandas as pd
import keybert
from nltk.corpus import stopwords
import re 


c:\Users\Marcell Suyanto\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
load_dotenv()
user = os.getenv("UID")
password = os.getenv("PASSWORD")

#JOB TYPE: 
# Graduate 
# Internsip 
# Temporary 
# Summer
job_type = "Internship"

<h2>Logging into the Job Portal</h2>

In [3]:
driver = webdriver.Chrome()
driver.get("https://www.cedars.hku.hk/netjobs")
main_page = driver.current_window_handle

# Click on Student Login
student_login = driver.find_element(By.XPATH, "//a[text()='HKU Student']")
student_login.click()

#Input email and log in
email_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "email"))
)
login_button = driver.find_element(By.ID, "login_btn")

email_input.send_keys(user)
login_button.click()

#Input password
password_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "passwordInput"))
)
sign_in_button = driver.find_element(By.ID, "submitButton")
password_input.send_keys(password)
sign_in_button.click()

#Trust Page
continue_button = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "idSIButton9"))
)
continue_button.click()
#Stay Singed in page
stay_button = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='submit' and @id='idSIButton9']"))
)
stay_button.click()

check_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='checkbox']"))
)
check_box.click()
agree_btn = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.ID, "btn-agree"))
)
agree_btn.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

In [4]:

yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')
yesterday

'2025-08-17'

In [5]:
def clean_data(data):
    # data: list of strings corresponding to job details
    for i in range(len(data)):
        data[i] = data[i].split(':\n')
    return data

def get_data(driver):
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    for window_handle in driver.window_handles:
        if window_handle != main_page:
            driver.switch_to.window(window_handle)
            break
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='content']"))
    )
    extract_data = driver.find_elements(By.XPATH, "//div[@id='content']//div[contains(@class, 'crow')]")
    data = [i.text for i in extract_data if i.text.strip()]
    
    driver.close()
    driver.switch_to.window(main_page)
    return data

details = []
jobs = driver.find_elements(By.XPATH, "//table[@id='search_jobs']/tbody/tr")

for i, job in enumerate(jobs):
    job.click()
    data = clean_data(get_data(driver))
    details.append(data)

driver.close()

In [6]:
details[2]

[['Job ID', 'G2500346'],
 ['Posting Date', '2025-08-14'],
 ['Employment Type', 'Internship'],
 ['Employment Mode', 'Full-time'],
 ['Company Name (Eng)', 'Syfe Hong Kong Limited'],
 ['Nature of Business', 'Banking / Financial Services'],
 ['Origin of Company Ownership', 'Hong Kong'],
 ['No. of Employees', '100-500 employees'],
 ['Position Offered', 'Marketing Intern'],
 ['No. of Vacancies', '1'],
 ['Job Nature', 'Advertising / Marketing / Marketing Research'],
 ['Job Description',
  'Support customer lifecycle management of marketing initiatives.\nCreate engaging ads to drive acquisition.\nConduct competitive analysis to inform strategy.\nDevelop compelling content for various channels.\nCollaborate with KOLs and strategic partners.\nWork closely with local and regional teams.\nAssist with ad-hoc and administrative tasks as needed.'],
 ['Basic Salary', 'HKD 15000 per Month'],
 ['Other Benefits', 'WFH on Fridays'],
 ['Work Location', 'Hong Kong'],
 ['Work Mode', 'Hybrid work'],
 ['Fields

In [7]:
job_details = dict()
for job in details:
    for detail in job:
        if detail[0] not in job_details:
            job_details[detail[0]] = 1
        else:
            job_details[detail[0]] += 1

job_details


{'Job ID': 20,
 'Posting Date': 20,
 'Employment Type': 20,
 'Employment Mode': 20,
 'Company Name (Eng)': 20,
 'Nature of Business': 20,
 'Origin of Company Ownership': 20,
 'No. of Employees': 20,
 'Position Offered': 20,
 'No. of Vacancies': 20,
 'Job Nature': 20,
 'Job Description': 20,
 'Basic Salary': 20,
 'Work Location': 20,
 'Work Mode': 20,
 'Fields of Study Required': 20,
 'Level of Award ': 20,
 'No. of working hours per day': 20,
 'No. of working days per week': 20,
 'Closing Date for Application': 20,
 'Application should be submitted by/via:': 20,
 'Email': 17,
 'Application Documents Required': 20,
 'Other Benefits': 5,
 'Application should be addressed to': 13,
 'Online via NETjobs:': 2,
 'Online Application at company website': 3,
 'Additional Information': 1,
 'Company Name (Chi)': 7}

In [8]:
jobs_data = []
for job in details:
    jobs_dict = dict()
    for detail in job:
        if len(detail) == 2: 
            jobs_dict[detail[0]] = detail[1]
        elif len(detail) > 2:  
            jobs_dict[detail[0]] = ' '.join(detail[1:])
        elif len(detail) == 1:
            if detail[0] not in jobs_dict:
                jobs_dict[detail[0]] = 1
            else:
                jobs_dict[detail[0]] += 1
    jobs_data.append(jobs_dict)
jobs_df = pd.DataFrame(jobs_data)

In [9]:
# jobs_df.to_json("data.json", orient="records", indent=4)

jobs_df.columns

Index(['Job ID', 'Posting Date', 'Employment Type', 'Employment Mode',
       'Company Name (Eng)', 'Nature of Business',
       'Origin of Company Ownership', 'No. of Employees', 'Position Offered',
       'No. of Vacancies', 'Job Nature', 'Job Description', 'Basic Salary',
       'Work Location', 'Work Mode', 'Fields of Study Required',
       'Level of Award ', 'No. of working hours per day',
       'No. of working days per week', 'Closing Date for Application',
       'Application should be submitted by/via:', 'Email',
       'Application Documents Required', 'Other Benefits',
       'Application should be addressed to', 'Online via NETjobs:',
       'Online Application at company website', 'Additional Information',
       'Company Name (Chi)'],
      dtype='object')

In [10]:
jobs_df.to_csv("jobs_data.csv", index=False)

In [ ]:
test = pd.read_csv("jobs_data.csv")
test.head()

,Job ID,Posting Date,Employment Type,Employment Mode,Company Name (Eng),Nature of Business,Origin of Company Ownership,No. of Employees,Position Offered,No. of Vacancies,...,Closing Date for Application,Application should be submitted by/via:,Email,Application Documents Required,Other Benefits,Application should be addressed to,Online via NETjobs:,Online Application at company website,Additional Information,Company Name (Chi)
0,G2500350,2025-08-15,Internship,Part-time,Grand Alliance Asset Management Limited,Banking / Financial Services,Hong Kong,10-50 employees,Data Analyst / Developer Intern,1,...,2025-10-20,1,cfcheng@gaamhk.com,"CV, Cover Letter",NaN,NaN,NaN,NaN,NaN,NaN
1,G2500349,2025-08-15,Internship,Part-time,"WealthRyse (Hong Kong) Co., Limited",Banking / Financial Services,South Korea,10-50 employees,Social Media & Visuals Analyst,2,...,2025-08-31,1,hr.external@wealthryse.ai,"CV, Cover Letter, Transcript",Meal Allowance,HR (HR),NaN,NaN,NaN,NaN
2,G2500346,2025-08-14,Internship,Full-time,Syfe Hong Kong Limited,Banking / Financial Services,Hong Kong,100-500 employees,Marketing Intern,1,...,2025-09-30,1,cody.ng@syfe.com,CV,WFH on Fridays,Cody Ng (Marketing Director),NaN,NaN,NaN,NaN
3,G2500345,2025-08-14,Internship,Part-time,ESD Services Limited,Information Technology / Telecom / Technology,Hong Kong,100-500 employees,Internship,1,...,2025-09-30,1,NaN,CV,NaN,NaN,1.0,https://hecl.taleo.net/careersection/esd+exter...,NaN,NaN
4,G2500335,2025-08-13,Internship,Part-time,"WealthRyse (Hong Kong) Co., Limited",Banking / Financial Services,South Korea,10-50 employees,Business Analyst,1,...,2025-08-31,1,hr.external@wealthryse.ai,"CV, Cover Letter, Transcript",Meal Allowance,HR (HR),NaN,NaN,NaN,NaN


: 

In [11]:
print(jobs_df['Job Description'].iloc[19])

"""
About Us
Role Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities
Key Responsibilities
Requirements/Qualifications
"""

Oxbridge Economics is seeking a Marketing Associate who is creative, organized, and passionate about bridging strategic communication with compelling design. You will support our business development and client engagement efforts by producing high-quality marketing materials, coordinating events, and serving as a key liaison with potential clients and partners.
  Key Responsibilities Client Communication: Engage with prospective clients, partners, and stakeholders through email, meetings, and events to promote Oxbridge’s offerings.
Marketing Content Creation:Design and prepare decks, posters, email campaigns, social media posts, and promotional videos to support product launches and brand visibility.
Event Coordination:Organize and execute marketing events, webinars, and campaigns, including logistics, materials, and follow-up communication.
  Qualifications    Strong communication skills, both written and verbal, in English.
 Proven ability to create polished marketing materials with 

'\nAbout Us\nRole Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities\nKey Responsibilities\nRequirements/Qualifications\n'

In [12]:
def clean_desc(desc):
    desc = re.split(r'', desc)
    return desc

In [13]:
def get_tags(input_df:pd.DataFrame, model:keybert) -> list:
    tags = []
    job_nature = input_df['Job Nature'].replace('/', ' ') 
    #job_desc = clean_desc(input_df['Job Description'])
    job_desc = input_df['Job Description']
    pos_offered = input_df['Position Offered'] 
    nature_of_business = input_df['Nature of Business'] 

    tags.append(model.extract_keywords(job_desc, stop_words=stopwords.words('english')))
    return tags

In [14]:
model = keybert.KeyBERT()

In [15]:
input_data = jobs_df[['Job ID', 'Job Nature', 'Job Description', 'Position Offered', 'Nature of Business']]
print(input_data.iloc[1]['Job Description'])
print(get_tags(input_data.iloc[1], model))

Job Description
Position Summary We are seeking a dynamic and creative intern to join our team to support both social media content creation and PowerPoint presentation design. This role is ideal for someone with a keen eye for visual storytelling, a passion for branding, and a strong grasp of digital communication trends. You will work closely with our marketing and client-facing teams to elevate our brand presence and enhance the visual impact of our presentations.

Key Responsibilities Design and refine PowerPoint presentations for internal and external use, ensuring visual consistency and brand alignment.
Create engaging social media content (graphics, captions, short videos) across platforms such as LinkedIn, Instagram, and X.
Collaborate with team members to translate complex ideas into clear, compelling visuals.
Assist in developing visual templates and style guides for presentations and digital assets.
Monitor social media trends and suggest creative ways to improve engagement.

Job ID  
Company Name  
Nature of Business  
Position Offered  
Job Nature
Job Description

Get hastags and concatenate the vector embeddings  
Store each embedding to its respective job ID

Data types: NULL INTEGER REAL TEXT BLOB

In [16]:
#Set up a connection
# conn = sqlite3.connect('jobs.db')
# cursor = conn.cursor()
# create_table_query = """
# CREATE TABLE IF NOT EXISTS jobs (
#     JobID TEXT PRIMARY KEY,
#     PostingDate TEXT,
#     EmploymentType TEXT,
#     EmplymentMode TEXT,
#     CompanyNameEng TEXT,
#     CompanyNameChi TEXT, 
#     NatureOfBusiness TEXT,
#     OriginOfCompanyOwnership TEXT,
#     NoOfEmployees TEXT,
#     PositionOffered TEXT,
#     NoOfVacancies TEXT,
#     JobNature TEXT,
#     JobDescription TEXT,
#     BasicSalary TEXT,
#     WorkLocatin TEXT, 
#     WorkMode TEXT,
#     FieldsOfStudyRequired TEXT,
#     LevelOfAward TEXT,
#     NoOfWorkingHoursPerDay INTEGER,
#     NoOfWorkingHoursPerWeek INTEGER,
#     ClosingDataForApplication TEXT,
#     ApplicationShouldBeSubmittedVia TEXT,
#     ApplicationDocumentsRequired TEXT
# );
# """

# cursor.execute(create_table_query)

In [17]:


import nbformat

# Load the notebook
with open('get_jobs.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Extract code cells
code_cells = [cell.source for cell in notebook.cells if cell.cell_type == 'code']

# Write to a .py file
with open('output_script.py', 'w') as f:
    for cell in code_cells:
        f.write(cell + '\n\n')
